In [15]:
import numpy as np
import matplotlib.pyplot as plt
import control as ct
from numpy import array
import scienceplots

np.set_printoptions(precision=3, suppress=True)

plt.style.use([
    'grid',
    'retro'
])

In [16]:
Ms = 1500
Mu = 150

Ks = 3000
Ku = 30000

Cs = 500

M = np.diag([Ms, Mu])

K = np.array([
    [Ks+Ku, -Ks],
    [-Ks, Ks]
])

C = np.array([
    [Cs, -Cs],
    [-Cs, Cs]
])

N = M.shape[0]

In [17]:
_M = -np.linalg.inv(M)

A = np.block([
    [np.zeros((N, N)), np.eye(N)],
    [_M@K, _M@C]
])

F = np.array([
    [0,  0],
    [Ks, Ku],
])

B = np.block([
    [np.zeros((N, N))],
    [-_M@F]
])

_C = np.eye(2*N)
D = np.zeros((2*N, N))

sys = ct.ss(A, B, _C, D)
tf_sys = ct.ss2tf(sys)

In [18]:
B

array([[  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [ 20., 200.]])

In [19]:
%matplotlib qt

ct.bode(tf_sys[0,0], omega_limits=(1e-1, 1e3))
ct.bode(tf_sys[2,0], omega_limits=(1e-1, 1e3))

plt.legend(['$x_u$', '$x_s$'])

In [26]:
T = np.arange(0, 30, 1e-2)
U = np.zeros((2, T.shape[0]))

#U[0, :] = np.sin(2*np.pi*.1*T)
#U[1, :] = np.sin(2*np.pi*.2*T)

_,y = ct.forced_response(
    sys, T, U, X0=np.random.randn(4, 1)
)

plt.plot(T, y[0])
plt.plot(T, y[1])


In [28]:
x1 = y[0]
x2 = y[1]

x1_dot = y[2]
x2_dot = y[3]

# Energia Cinetica
E = (1/2) * (Ms*x1_dot**2 + Mu*x2_dot**2)

# Energia Potencial
V = (1/2) * (Ku*(x1-x2)**2 + Ks*x2**2)

# Energia Total
Te = E + V

# # # # #

MI = np.vectorize(lambda x: 1/np.sqrt(x) if x != 0 else 0)(M)
Kt = MI@K@MI

wn, P = np.linalg.eig(Kt)
wn = np.sqrt(wn)

wn /= 2*np.pi

# for each eigenvalue ( natural frequency ), plot an bar
# of the energy of the system at that frequency with the eigenvector

%matplotlib qt

colors = ['C1', 'C2', 'C3', 'C4']

for i in range(len(wn)):
    w = wn[i]
    p = np.abs(P[:, i])**2

    for j in range(len(p)):
        plt.bar(w, p[j], bottom=np.sum(p[:j]), width=.01, color=colors[j])


plt.xlabel('Frequência (Hz)')
plt.ylabel('Energia')
# y ticks in percentage
plt.gca().yaxis.set_major_formatter(lambda x, _: f'{x*100:.0f}%')

plt.legend([f'${i+1}$' for i in range(N)])

In [22]:
plt.figure()
plt.plot(T,E)
plt.plot(T,V)

In [23]:
w     = np.linspace(1e-1, 10, 1000)
zetas = np.linspace(.1, 1, 6)

epsilon = Ms/Mu
ws      = np.sqrt(Ks/Ms)
wu      = np.sqrt(Ku/Mu)
alpha   = ws/wu
r       = w/ws

fig, axs = plt.subplots(3, 1, sharex=True, figsize=(10, 10))

# axs[0] = mu: Xs/Y transmissibility suspended mass
# axs[1] = tau: Xu/Y transmissibility unsuspended mass
# axs[2] = eta: Z/Y displacement transmissibility

for zeta in zetas:
    Z1 = (r**2*(r**2*alpha**2-1)+(1-(1+epsilon)*r**2*alpha**2))
    Z2 = 2*zeta*r*(1-(1+epsilon)*r**2*alpha**2)

    mu  = np.sqrt( (4*zeta**2*r**2 + 1)/(Z1**2+Z2**2) )
    tau = np.sqrt( (4*zeta**2*r**2+1+r**2*(r**2-2))/(Z1**2+Z2**2) )
    eta = np.sqrt(r**4 / (Z1**2+Z2**2))

    axs[0].plot(w, mu,  label=f'$\\zeta={zeta:.1f}$')
    axs[1].plot(w, tau, label=f'$\\zeta={zeta:.1f}$')
    axs[2].plot(w, eta, label=f'$\\zeta={zeta:.1f}$')

# put the Y label in diagonal to save space

axs[0].set_ylabel('$\mu = \\frac{X_s}{y}$')
axs[1].set_ylabel('$\\tau = \\frac{X_u}{Y}$')
axs[2].set_ylabel('$\eta = \\frac{Z}{Y}$')
axs[2].set_xlabel('$r$')

plt.legend()
plt.savefig('transmissibility.png', dpi=300, bbox_inches='tight')

In [24]:
# print(f'epsilon = {epsilon:.3f}')
# print(f'ws = {ws/2/np.pi:.3f}')
# print(f'wu = {wu/2/np.pi:.3f}')
# print(f'alpha = {alpha:.3f}')